# Strictness and laziness
A function is called _non-strict_ if it may choose _not_ to evaluate one of its arguments. As an example, function `&&` takes two arguments but only evaluates the second argument if the first evaluates to `true`. This function could be called _strict in its first argument_ but _non-strict in the second_. Here's a non-strict `if2`:

In [ ]:
def if2[A](cond: Boolean, onTrue: => () => A, onFalse: () => A) =
    if (cond) onTrue() else onFalse()

val res = if2(false, 
            () => throw new Exception("Got true"), 
            () => "Got false")


Formally, a function `f` is strict if the expression `f(x)` evalutes to bottom (does not terminate) for all `x` that evaluate to `bottom`. An expression evaluates to `bottom` if it throws an error or runs forever instead of returning a definite value. 

The unevaluated form of an expression is called a _thunk_. Callers of the `if2` function explicitly need to create thunks, i.e., functions is empty argument list. Scala offers prettier syntax:

In [ ]:
def if2[A](cond: Boolean, onTrue: => A, onFalse: => A): A = if(cond) onTrue else onFalse

if2(false, sys.error("Got exception"), "Got false") // Does not require `() => "Got false"`!

The thunk is re-evaluated by default every time it's referenced:

In [ ]:
def maybeTwice(b: Boolean, i: => Int): Int = if (b) i+i else 0

maybeTwice(true, { println("Evaluating"); 1+41 }) // Prints "Evaluating" twice!

The result of the evaluation can be cached using the `lazy` keyword:

In [ ]:
def maybeTwice2(b: Boolean, i: => Int): Int = {
    lazy val j = i // Caches the result, delays execution until `i` is first referenced
    if (b) j+j else 0
}
maybeTwice2(true, { println("Evaluating"); 1+41 }) // Prints "Evaluating" once!

### Lazy lists i.e. streams

In [ ]:
sealed trait Stream[+A]
case object Empty extends Stream[Nothing]
case class Cons[+A](h: () => A, t: () => Stream[A]) extends Stream[A] // Constructor arguments cannot use `: =>` syntax

object Stream {
    // "Smart" constructor with nicer arguments and caching of evaluations
    def cons[A](hd: => A, tl: => Stream[A]): Stream[A] = {
        lazy val head = hd // Caches the result of evaluating `hd` so that it is only evaluated once
        lazy val tail = tl
        Cons(() => head, () => tail)
    }

    def empty[A]: Stream[A] = Empty // Helps type inference later on
    
    def apply[A](as: A*): Stream[A] = 
        if (as.isEmpty) Empty else cons(as.head, apply(as.tail: _*))
}

val testStream: () => Stream[Int] = () => Stream.cons(1, Stream.cons({ print("Evaluating second"); 2 }, Stream.cons({ sys.error("fail"); 3}, Stream.empty)))

Definitions are similar as for `List` (Chapter 3) except that the constructor `Cons` takes explicit _thunks_ that do not need to be evaluted. Extracting values from the stream requires forcing the evaluation:

In [ ]:
def headOption[A](a: Stream[A]): Option[A] = 
    a match {
        case Empty => None
        case Cons(h, t) => Some(h()) // Forces evaluation
    }

// Using the smart constructor does not require explicit definition of thunks with `() => {}`
val stream = Stream.cons({ println("Evaluating..."); 3 }, Empty)

headOption(stream)
headOption(stream) // Does not print "Evaluating" as the value is cached in smart constructor

### Exercise 5.1
Write a function to convert a `Stream` to `List`.

In [ ]:
def toList[A](a: Stream[A]): List[A] = {
    a match {
        case Empty => List()
        case Cons(h, t) => h() :: toList(t())
    }
}

toList(Stream(1, 2, 3))

### Exercise 5.2
Write the function `take(n)` for returning the first `n` elements of a `Stream`, and `drop(n)` for skipping the first `n` elements of a `Stream`.

In [ ]:
def take[A](stream: Stream[A], n: Int): Stream[A] = {
    if (n <= 0) {
        Stream.empty
    } else {
        stream match {
            // This neither evaluates the expression in `h` nor `t` as `Stream.cons` defines its argument with `h: => A`
            // `Cons(h, t) => Cons(h, () => take(t(), n -1))` would also work
            case Cons(h, t) => Stream.cons(h(), take(t(), n - 1))
            case Empty => Stream.empty
        }
    }
}

def drop[A](stream: Stream[A], n: Int): Stream[A] = {
    if (n <= 0) {
        stream
    } else {
        stream match {
            case Cons(h, t) => drop(t(), n-1)
            case Empty => Stream.empty
        }
    }
}

val streamWithErrorAtHead = Stream.cons({ sys.error("fail") }, Stream.cons(2, Empty))
val streamWithErrorAtTail = Stream.cons(2, Stream.cons({ sys.error("fail") }, Empty))

toList(drop(streamWithErrorAtHead, 1)) // List(2)
toList(take(streamWithErrorAtTail, 1)) // List(2)

### Exercise 5.3
Write the function `takeWhile`.

In [ ]:
def takeWhile[A](stream: Stream[A], p: A => Boolean): Stream[A] = {
    stream match {
        case Cons(h, t) => if (p(h())) Stream.cons(h(), takeWhile(t(), p)) else Empty
        case Empty => Empty
    }
}

val stream = Stream.cons(1, Stream.cons({ print("Hi"); 2 }, Stream.cons({ sys.error("fail"); 3}, Stream.empty)))
val shortStream = takeWhile(stream, (n: Int) => n < 2) // Does not print anything
val evaluateStream = toList(shortStream) // Prints "Hi"

Streams allow separating the concern of describing an expression from actually evaluating it. For example, _early termination_  is a breeze with streams:

In [ ]:
def exists[A](stream: Stream[A], p: A => Boolean): Boolean = 
    stream match {
        case Cons(h, t) => p(h()) || exists(t(), p) // Does not touch the tail stream if finds match
        case Empty => false
    }

exists(testStream(), (a: Int) => a == 2)

In [ ]:
def foldRight[A,B](stream: Stream[A], z: => B)(f: (A, => B) => B): B = 
    stream match {
        case Cons(h, t) => f(h(), foldRight(t(), z)(f))
        case Empty => z
    }

foldRight(Stream(1, 2, 3), 0)(_ + _)

In [ ]:
def exists[A](stream: Stream[A], p: A => Boolean): Boolean = 
    foldRight(stream, false)((a, b) => p(a) || b)

val stream = Stream.cons(1, Stream.cons({ print("Evaluating second"); 2 }, Stream.cons({ sys.error("fail"); 3}, Stream.empty)))
exists(stream, (a: Int) => a == 2)

### Exercise 5.4
Implement `forAll` that should terminate as soon as it encounters a nonmatching value.

In [ ]:
def forAll[A](stream: Stream[A])(p: A => Boolean): Boolean = 
    foldRight(stream, true)((a, b) => p(a) && b)

forAll(testStream())(a => a < 2)

### Exercise 5.5
Use `foldRight` to implement `takeWhile`.

In [ ]:
def takeWhile[A](stream: Stream[A])(p: A => Boolean): Stream[A] =
    foldRight(stream, Stream.empty: Stream[A])((a, b) => if (p(a)) Stream.cons(a, b) else Stream.empty)

toList(takeWhile(testStream())(a => a < 2))

### Exercise 5.7
Implement `map`, `filter`, `append`, and `flatMap` using `foldRight`.